# Proyecto final Machine Learning

Jorge Ruizvisfocri

Emilio Martinez

# Descripción del proyecto

El siguiente proyecto busca clasificar imágenes de Carcinomas Ductales Invasivos (IDC en inglés) extraidas de muestras de pacientes con cancer de mama.

La base de datos fue tomada de https://www.kaggle.com/datasets/paultimothymooney/breast-histopathology-images

Se propone utilizar una red neuronal convolucional para resolver el problema de clasificación.

# Paqueterías de trabajo

In [1]:
## Paquetes de ciencias de datos
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import math
import random

## Paquetes de lectura de imagenes
from pathlib import Path
import os
import glob

## Paquetes de imágenes
from PIL import Image
import cv2

##Otros
from tqdm import tqdm

# Datos de trabajo

In [2]:
# Directorio de imágenes
data_dir = Path("D://bases de datos//proyecto_ml_final//imagenes") ## Directorio de datos

Dado que todos los pacientes tienen muestras con tumores malignos y benignos, la aleatorización puede realizarse a nivel de imagen o a nivel de paciente.

Bajo el supuesto de que podría existir correlación de algún tipo entre las imágenes pertenecientes a un mismo paciente, creemos que sería interesante aleatorizar a nivel de paciente, para introducirle a la red la información de personas que jamás ha visto.

In [3]:
### Aleatorización a nivel de paciente
#### Obtenemos la lista de pacientes
px = [f for f in data_dir.iterdir() if f.is_dir()]

In [4]:
### Obtenemos el número equivalente al porcentaje deseado
k = math.ceil( len(px) * 80 // 100)

In [ ]:
### Hacemos la muestra de pacientes para entrenamiento y prueba

In [5]:
### fijamos la semilla para tener reproductibilidad
random.seed(4352)

In [6]:
train_id = random.sample(px,k)

In [7]:
test_id = set(px) - set(train_id)

In [8]:
### Limpieza
del data_dir
del px
del k

In [9]:
### cargamos files de entrenamiento
train_images_files = []
for p in tqdm(train_id):
    img_lst = list(p.rglob("*.png"))
    train_images_files.extend(img_lst)

100%|████████████████████████████████████████████████████████████████████████████████| 223/223 [00:32<00:00,  6.80it/s]


In [10]:
test_images_files = []
for p in tqdm(test_id):
    img_lst = list(p.rglob("*.png"))
    test_images_files.extend(img_lst)

100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [00:09<00:00,  6.10it/s]


In [11]:
### Revisamos que las direcciones de las imágenes estén bien cargadas
print(
    f"Número de imagenes\n"
    "-----------------\n"
    f"Total: {len(train_images_files) + len(test_images_files) }\n"  # 277,524 tiles
    "-----------------\n"
    f"Entrenamiento: {len(train_images_files) }\n"
    "-----------------\n"
    f"Prueba: {len(test_images_files) }\n"
)

Número de imagenes
-----------------
Total: 277524
-----------------
Entrenamiento: 220389
-----------------
Prueba: 57135



In [12]:
### Lipieza
del train_id
del test_id

## Preparamos las etiquetas

In [13]:
## y_entrenamiento
### Preparamos las etiquetas
y_train = []
for name in tqdm(train_images_files):
    etiqueta = str(name)[-5]
    etiqueta_num = int(etiqueta)
    y_train.append(etiqueta_num)

100%|██████████████████████████████████████████████████████████████████████| 220389/220389 [00:00<00:00, 223263.86it/s]


In [14]:
y_test = []
for name in tqdm(test_images_files):
    etiqueta = str(name)[-5]
    etiqueta_num = int(etiqueta)
    y_test.append(etiqueta_num)

100%|████████████████████████████████████████████████████████████████████████| 57135/57135 [00:00<00:00, 200750.89it/s]


In [19]:
### Revisamos números de positivos en conjuntos
print(
    f"Positivos totales\n"
    "-----------------\n"
    f"Total: {sum(y_train) + sum(y_test)}\n"  # 277,524 tiles
    "-----------------\n"
    f"Entrenamiento: {sum(y_train) }\n"
    "-----------------\n"
    f"Prueba: {sum(y_test) }\n"
     "------------------------\n"
    f"Positivos porcentajes\n"
    "-----------------\n"
    f"Total: { round((sum(y_train) + sum(y_test))/ (len(y_train) + len(y_test)),2)  }\n"  # 277,524 tiles
    "-----------------\n"
    f"Entrenamiento: {round(sum(y_train)/ len(y_train),2) }\n"
    "-----------------\n"
    f"Prueba: {round(sum(y_test)/ len(y_test),2) }\n"
)

Positivos totales
-----------------
Total: 78786
-----------------
Entrenamiento: 64466
-----------------
Prueba: 14320
------------------------
Positivos porcentajes
-----------------
Total: 0.28
-----------------
Entrenamiento: 0.29
-----------------
Prueba: 0.25



## Preparamos las imágenes

In [20]:
### Cargamos de imágenes de entrenamiento en una lista
dataset_img_train = list()
for img in tqdm(train_images_files):
    image = Image.open(img)
    image=image.resize((50,50))
    numpydata = np.asarray(image)
    dataset_img_train.append(numpydata)

100%|█████████████████████████████████████████████████████████████████████████| 220389/220389 [30:46<00:00, 119.34it/s]


In [21]:
### Convertimos la lista en tensor
dataset_img_train_array = np.asarray(dataset_img_train,dtype=object)

In [22]:
### Limpieza
del train_images_files
del etiqueta
del etiqueta_num
del dataset_img_train
del numpydata

In [23]:
dataset_img_train_array.shape ## Debe darnos un vector de tamaño (n,a,b,c)

(220389, 50, 50, 3)

In [24]:
### Aplanamos los datos
dataset_img_train_array_plana = dataset_img_train_array.astype('float32') / 255

MemoryError: Unable to allocate 6.16 GiB for an array with shape (220389, 50, 50, 3) and data type float32

In [ ]:
dataset_img_train_array_plana[0:3] ## Vemos las primeras entradas para verificar que estén aplanados los datos

In [ ]:
del dataset_img_train_array

In [ ]:
## Revisamos que tengan la misma longitud para el conjunto de entrenamiento
leny_train = len(y_train)
lenset_train = len(dataset_img_train_array_plana)

if leny_train == lenset_train:
    print("El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de entrenamiento")
else:
    print("El tamaño del vector de resultados y las imágenes no coincide en el conjunto de entrenamiento")

In [ ]:
### Cargamos imágenes de prueba en una lista
dataset_img_test = list()
for img in tqdm(test_images_files):
    image = Image.open(img)
    image=image.resize((50,50))
    numpydata = np.asarray(image)
    numpydata_plana = numpydata.astype('float32') / 255
    dataset_img_test.append(numpydata_plana)

In [ ]:
### Convertimos la lista en tensor
dataset_img_test_array = np.asarray(dataset_img_test,dtype=object)

In [ ]:
### Limpieza
del test_images_files
del dataset_img_test
del image
del numpydata
del numpydata_plana

In [ ]:
dataset_img_test_array.shape ## Debe darnos un vector de tamaño (n,a,b,c)

In [ ]:
### Aplanamos los datos
dataset_img_test_array_plana = dataset_img_test_array.astype('float32') / 255

In [ ]:
dataset_img_test_array_plana[0:3] ## Vemos las primeras entradas para verificar que estén aplanados los datos

In [ ]:
## Revisamos que tengan la misma longitud para el conjunto de prueba
leny_test = len(y_test)
lenset_test = len(dataset_img_test_array_plana)

if leny_test == lenset_test:
    print("El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de prueba")
else:
    print("El tamaño del vector de resultados y las imágenes no coincide en el conjunto de prueba")

# Modelo

## Conjunto de entrenamiento y validación

In [ ]:
### Creamos conjuntos de validación y entrenamiento
X_train, X_val, y_train, y_val = train_test_split(dataset_img_train_array_plana, y_train, test_size=0.33, random_state=42)

In [ ]:
### Limpieza
del dataset_img_train_array_plana

## Arquitectura del modelo

In [ ]:
### Usamos modelo de la tarea 11
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same', input_shape=(50, 50, 3)))
model.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
#model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
#model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(Adam(learning_rate=.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

## Entrenamiento del modelo

In [ ]:
## Entrenamos el modelo
history = model.fit(X_train, y_train, batch_size=256, epochs=2, validation_data=(X_val, y_val), shuffle=True)

## Gráficas de pérdida y precisión en el entrenamiento

### Pérdida

In [ ]:
for hist, lr in zip(history, learning_rates):
    loss = hist.history['loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'o-', label=lr)
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Precisión

In [ ]:
for hist, lr in zip(history, learning_rates):
    loss = hist.history['accuracy']
    epochs = range(1, len(loss) + 1)
    plt.semilogy(epochs, loss, 'o-', label=lr)
plt.title('Training accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Predicciones

In [ ]:
pred = model.predict(dataset_img_test_array_plana)
print(classification_report(y_test, np.argmax(pred,axis=1)))